In [23]:
from metadata_builder import *
import pandas as pd
from datetime import datetime

In [24]:
filename = 'employees'
data_path = '../data/' + f'{filename}.csv'
df = pd.read_csv(data_path)

In [27]:
datetime.now().isoformat()

'2025-10-15T00:58:05.970053'

In [25]:
df

,Emp_ID,Name,Gender,Age,City,Department,Salary,Email
0,1,Tariq,Male,47.0,Karachi,Sales,123609.72,tariq76@gmail.com
1,2,Hina,Female,26.0,Quetta,HR,125683.94,hina90@gmail.com
2,3,Usman,Male,36.0,Lahore,Sales,NaN,usman71@gmail.com
3,4,Sana,Female,21.0,Quetta,Finance,147901.41,sana72@gmail.com
4,5,Ayesha,Male,40.0,lahore,HR,123880.80,ayesha25@gmail.com
5,6,Ahmed,Male,NaN,Quetta,Finance,50115.70,ahmed33@gmail.com
6,7,Bilal,Female,60.0,Peshawar,Finance,32984.18,bilal93@gmail.com
7,8,Ahmed,Female,NaN,Quetta,IT,NaN,ahmed89@gmail.com
8,9,Usman,Male,NaN,Quetta,Sales,NaN,usman35@gmail.com
9,10,Sara,Female,NaN,Islamabad,Finance,127273.37,sara73@gmail.com


In [26]:
api_key = "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"
timestamp = datetime.now().isoformat().replace(":", "-")
builder = RSIMetadataBuilder(api_key=api_key, model_name="openai/gpt-4o", kb_path=f"../config/metadata_kb_{filename}_{timestamp}.json")
first_order = builder.build_first_order_metadata(df)

enriched = builder.ai_augment_metadata(first_order)
builder.save_metadata(enriched, file_path=f'../config/metadata_enriched_{filename}_{timestamp}.json')

#Example of manual feedback
# builder.update_feedback("amt", {
    # "semantic_type": "currency",
    # "possible_meaning": "Transaction amount in AUD",
    # "expected_format": "float",
    # "potential_issues": "Missing or negative values"
# })
# 

Metadata saved to ../config/metadata_enriched_employees_2025-10-15T00-57-16.875259.json


NameError: name 'random' is not defined